# Homework 4 Solutions

1. Download Daily Bars for FB, AAPL, AMZN, NFLX, GOOGL and QQQ from yahoo finance starting 2016-01-01. Use the Adj Close to compute daily returns.

In [17]:
import yfinance as yf
import numpy as np
import pandas as pd

univ = ['META','AAPL','AMZN','NFLX','GOOGL','QQQ']
px = yf.download(univ, start="2020-01-01", auto_adjust=False)['Adj Close']
ret = px / px.shift() - 1

[*********************100%***********************]  6 of 6 completed


2. Now, let's compute the beta of FB, AAPL, AMZN, NFLX, GOOGL using QQQ as our benchmark. You can think of this as the beta these stocks have to their industry (tech). In practice,  we have to use some lookback window to compute the beta. Let's use 252 (1 year, excluding wknds/holidays). So, for each day, the betas should be computed using the most recent 252 data points.

While it's possible to use a loop + statsmodels OLS function here, the computation is simple enough we don't need that complexity and overhead. The formula for beta in a univariate regression (with a constant) is correlation*(vol_y/vol_benchmark). Here I computed this quickly using the pandas rolling function.

In [18]:
corr = ret.rolling(252).corr(ret['QQQ'])
vol = ret.rolling(252).std()
beta = (corr*vol).divide(vol['QQQ'],axis=0)  
beta

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Date,,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-07-22,0.923697,1.111020,0.868760,1.131661,0.798682,1.0
2025-07-23,0.924863,1.111877,0.869369,1.133724,0.796152,1.0
2025-07-24,0.928061,1.116644,0.867352,1.132698,0.794251,1.0


3. Using the betas, compute an "alpha" on each day. This is also known as a "residual return".

In [19]:
resid = ret - beta.multiply(ret['QQQ'],0)
resid

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Date,,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-07-22,0.013817,-0.002230,0.011019,-0.005588,-0.030887,3.469447e-18
2025-07-23,-0.005385,-0.001467,-0.009767,0.007272,-0.014807,-3.469447e-18
2025-07-24,-0.003796,0.014882,0.008352,-0.000701,0.001692,-1.301043e-18


4. Compare the volatility of the residual returns to that of the original returns. What do you notice?

The residual returns generally have much lower volatility. This is because we've taken out the "beta component" which drives much of the stock returns.

In [20]:
vol = {}
vol['orig'] = ret.std()*np.sqrt(252)
vol['resid'] = resid.std()*np.sqrt(252)
vol = pd.DataFrame(vol).drop('QQQ')
vol

,orig,resid
Ticker,,
AAPL,0.324244,0.170506
AMZN,0.359637,0.217047
GOOGL,0.326517,0.203956
META,0.444968,0.311853
NFLX,0.446788,0.356441


5. Compare the pairwise correlations of the residual returns to that of the original returns. What do you notice?

The pairwise correlations of the residual returns in general are much lower between stocks. This is because we've taken out one of the major common forces (tides) moving these stocks. Also striking is the large drop in correlations with QQQ. The original returns are 0.6-0.8 correlated with QQQ, but the residual returns are nearly 0 correlated with QQQ. Note, however, that they are not perfectly 0 correlated. This is because in practice we have to estimate the beta using rolling windows rather than the full sample

In [21]:
ret.corr()

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Ticker,,,,,,
AAPL,1.000000,0.601293,0.636149,0.567802,0.446663,0.832770
AMZN,0.601293,1.000000,0.653725,0.629902,0.532988,0.776944
GOOGL,0.636149,0.653725,1.000000,0.638527,0.437339,0.788329
META,0.567802,0.629902,0.638527,1.000000,0.495791,0.720542
NFLX,0.446663,0.532988,0.437339,0.495791,1.000000,0.592691
QQQ,0.832770,0.776944,0.788329,0.720542,0.592691,1.000000


In [22]:
resid.corr()

Ticker,AAPL,AMZN,GOOGL,META,NFLX,QQQ
Ticker,,,,,,
AAPL,1.000000,-0.122899,-0.016119,-0.110312,-0.105747,0.040356
AMZN,-0.122899,1.000000,0.131565,0.141332,0.067637,-0.067334
GOOGL,-0.016119,0.131565,1.000000,0.135044,-0.065179,-0.020858
META,-0.110312,0.141332,0.135044,1.000000,0.108290,-0.043404
NFLX,-0.105747,0.067637,-0.065179,0.108290,1.000000,-0.058195
QQQ,0.040356,-0.067334,-0.020858,-0.043404,-0.058195,1.000000


6. Compute the information ratio for each of these stocks and compare that to the sharpe ratio

The IRs are in general lower than the SRs, suggesting most of the performance of these stocks was driven by their exposure to QQQ. AAPL performed the best looking at IR or SR and FB performed the worst. 

In [23]:
df = {}
df['IR'] = resid.mean() / resid.std()*np.sqrt(252)
df['SR'] = ret.mean() / ret.std()*np.sqrt(252)
df = pd.DataFrame(df).drop('QQQ')
df

,IR,SR
Ticker,,
AAPL,-0.133761,0.762715
AMZN,-0.318008,0.629366
GOOGL,0.260859,0.738032
META,0.296128,0.725425
NFLX,0.307545,0.743516


### Blah Blah Blah